In [1]:
import torch.nn as nn
import torch

In [2]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h

In [3]:
my_cell = MyCell()

In [4]:
x, h = torch.rand(3, 4), torch.rand(3, 4)

In [5]:
traced_cell = torch.jit.trace(my_cell, (x, h))

In [6]:
traced_cell

TracedModule[MyCell](
  original_name=MyCell
  (linear): TracedModule[Linear](original_name=Linear)
)

In [7]:
traced_cell(x, h)

tensor([[ 0.4238, -0.0524,  0.5719,  0.4747],
        [-0.0059, -0.3625,  0.2658,  0.7130],
        [ 0.4532,  0.6390,  0.6385,  0.6584]],
       grad_fn=<DifferentiableGraphBackward>)

In [8]:
traced_cell.graph

graph(%self : ClassType<MyCell>,
      %input : Float(3, 4),
      %h : Float(3, 4)):
  %1 : ClassType<Linear> = prim::GetAttr[name="linear"](%self)
  %weight : Tensor = prim::GetAttr[name="weight"](%1)
  %bias : Tensor = prim::GetAttr[name="bias"](%1)
  %6 : Float(4, 4) = aten::t(%weight), scope: MyCell/Linear[linear] # /home/jibin/.local/lib/python3.6/site-packages/torch/nn/functional.py:1370:0
  %7 : int = prim::Constant[value=1](), scope: MyCell/Linear[linear] # /home/jibin/.local/lib/python3.6/site-packages/torch/nn/functional.py:1370:0
  %8 : int = prim::Constant[value=1](), scope: MyCell/Linear[linear] # /home/jibin/.local/lib/python3.6/site-packages/torch/nn/functional.py:1370:0
  %9 : Float(3, 4) = aten::addmm(%bias, %input, %6, %7, %8), scope: MyCell/Linear[linear] # /home/jibin/.local/lib/python3.6/site-packages/torch/nn/functional.py:1370:0
  %10 : int = prim::Constant[value=1](), scope: MyCell # <ipython-input-2-c6e2cd8665ee>:7:0
  %11 : Float(3, 4) = aten::add(%9, %h, %10

In [9]:
traced_cell.code

'import __torch__\nimport __torch__.torch.nn.modules.linear\ndef forward(self,\n    input: Tensor,\n    h: Tensor) -> Tensor:\n  _0 = self.linear\n  weight = _0.weight\n  bias = _0.bias\n  _1 = torch.addmm(bias, input, torch.t(weight), beta=1, alpha=1)\n  return torch.tanh(torch.add(_1, h, alpha=1))\n'